In [1]:
%matplotlib agg

import copy

from utils.model import model_names_eps30, model_names_logr
from utils.ml import MLPNetwork, MOVotingRegressor
from utils.log import auto_log
from utils.vis_shap import summary_legacy
from utils.al import UniformCompositionGenerator


import shap
from shap.plots import colors
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import keras
from sklearn.utils import Bunch

In [2]:
models_eps30 = []  

for model_name in model_names_eps30:
    tf.keras.backend.clear_session()
    
    f_model = f'data/eps30_models/{model_name}.model'
    model = tf.keras.models.load_model(f_model)
    
    mlp_model = MLPNetwork.from_keras(model)
    mlp_model.name = model_name
    last_layer = model.layers[-1]
    if isinstance(last_layer, keras.layers.Rescaling):
        mlp_model.output_scale = last_layer.scale

    models_eps30.append(mlp_model)                

perf_model_eps30 = MOVotingRegressor([(m.name, m) for m in models_eps30])
names, clfs = perf_model_eps30._validate_estimators()
perf_model_eps30.estimators_ = [clf for name, clf in zip(names, clfs) if name != 'drop']
perf_model_eps30.named_estimators_ = Bunch(**{name: clf for name, clf in zip(names, clfs) if name != 'drop'})

In [4]:
import pickle

with open('svm_models.pkl', 'rb') as f:
    svm_model = pickle.load(f)

c_tags = [    'C_10',      'C_07',      'C_05',      'C_02']
c_nums = [        10,         7.5,           5,         2.5]
c_tags_lut = {k: v for k, v in zip(c_nums, c_tags)}

def feas_func(comps):
    return np.array([svm_model[c_tags_lut[row[3]]].predict([row[0:2]])[0] > 0.75 for row in comps])


In [5]:
train_df = pd.read_csv('data/eps30_data/eps_train_rd08.csv')
x_cols = ['CNF', 'MXN', 'GEL', 'Mass_loading', 'GA']
X_train, y_train_1 = train_df[x_cols], train_df['eps_30']
train_df


# Generate composition for the next round
g = UniformCompositionGenerator(materials=x_cols[:3], n_comps=30, n_iters=10000, random_state=1, 
                                existing_comps=train_df[x_cols],
                                perf_func=perf_model_eps30.predict_variance,
                                feas_func=feas_func,
                               )
comps = g.optimize()
comps = pd.DataFrame(comps, columns=x_cols)
comps

  0%|          | 0/10000 [00:00<?, ?it/s]

,CNF,MXN,GEL,Mass_loading,GA
0,0.912944,0.032073,0.054983,10.0,1.0
1,0.429485,0.372613,0.197902,5.0,0.0
2,0.218920,0.271839,0.509241,10.0,1.0
3,0.538920,0.371303,0.089777,5.0,1.0
4,0.448964,0.548352,0.002684,10.0,1.0
5,0.724805,0.047432,0.227763,10.0,1.0
6,0.289802,0.143561,0.566637,7.5,0.0
7,0.343008,0.614617,0.042375,5.0,1.0
8,0.341127,0.278611,0.380262,10.0,0.0
9,0.284665,0.470027,0.245308,10.0,0.0
